# 9.- Azure ML Deploy Training

In [2]:
import yaml

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml import Input, Output
from azure.ai.ml import command

## Define Variables

In [3]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [4]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

data_asset_path = config["train"]["data_asset_path"]

## Azure Authentication

In [5]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

In [6]:
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)

In [17]:
train_inputs = {
    "data": Input(type="uri_folder", path=data_asset_path),
    "model": Input(type="uri_file", path="../train/pretrained_models/yolo11n-seg.pt"),
    "epochs": 2,
    "batch": 2,
    "exist_ok": True,
    "device": "cpu"
    }

In [18]:
train_outputs = {
    "output_data": Output(type="uri_folder", path=data_asset_path)
    }

In [19]:
command_job = """cp ${{ inputs.data }}/data.yaml /tmp/data.yaml && \
sed -i "s|path:.*$|path: ${{ inputs.data }}|" /tmp/data.yaml && \
yolo segment train \
data= /tmp/data.yaml \
model= ${{inputs.model}} \
epochs= ${{inputs.epochs}} \
batch= ${{inputs.batch}} \
exist_ok= ${{inputs.exist_ok}} \
device= ${{inputs.device}} \
project= ${{outputs.output_data}} \
name= experiment"""

In [20]:
job = command(
    command = command_job,
    inputs= train_inputs,
    outputs= train_outputs,
    environment="docker-context-example-walter@latest",
    compute="training-gpu-cluster",
    experiment_name="yolo11-segmentation-training",
    display_name="yolo11-segmentation-training"
)

In [ ]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)